<a href="https://colab.research.google.com/github/lapatradaa/M-MMT4NL/blob/main/s_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai sentence-transformers pandas tqdm


In [2]:
import openai
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

In [3]:
INPUT_CSV = "/content/s-bert - original.csv"         # your input file (must have column 'original')
OUTPUT_CSV = "results.csv"
SBERT_THRESHOLD = 0.75

In [13]:
from openai import OpenAI
##client = OpenAI <“OPENAI_API_KEY”>

PROMPT_NEGATION = """You're an expert linguist in English and Thai.
You need to modify this Thai sentence by negating the sentence, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.
"I'm so tired" → "I'm so not energetic"
"I'm really hungry" → "I'm really not full"
"I'm not sure if I'm up for that" → "I'm sure I'm not up for that"
"I'm not sure if I can make it to the event" → "I'm unsure if I can make it to the event"
"I'm feeling a bit confused right now" → "I'm feeling a bit not clear right now"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.

Thai sentence: "{}"
"""

In [10]:
PROMPT_TAXONOMY = """You're an expert linguist in English and Thai. You need to modify this Thai sentence by substituting a word with its respective synonym, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.

I'll give you some examples of converting one sentence to another sentence: "I'm so tired" is converted to "I'm so exhausted" "I'm really hungry" is converted to "I'm really starving" "I'm not sure if I'm up for that" is converted to "I'm not certain if I'm up for that" "I'm not sure if I can make it to the event" is converted to "I'm not confident if I can make it to the event"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.

Thai sentence: "{}"
"""

def gpt_taxonomy(sentence: str) -> str:
    prompt = PROMPT_TAXONOMY.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            taxonomy = gpt_taxonomy(s)
            score = sbert_similarity(model, s, taxonomy)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "taxonomy": taxonomy,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "taxonomy": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)

Processing: 100%|██████████| 50/50 [00:50<00:00,  1.00s/it]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

In [12]:
PROMPT_NER = """Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.
You're an expert linguist in English and Thai. You need to modify this Thai sentence by replacing the named entity with a new name, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.

"I'm so tired" is converted to "Jane is so tired"
"I'm really hungry" is converted to "Jack is really hungry" "I'm not sure if I'm up for that" is converted to "Jones is not sure if she is up for that"
"I'm not sure if I can make it to the event" is converted to "Jill is not sure if she can make it to the event"
"I'm feeling a bit confused right now" is converted to "Andy is feeling a bit confused right now"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.

Thai sentence: "{}"
"""

def gpt_ner(sentence: str) -> str:
    prompt = PROMPT_NER.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            ner = gpt_ner(s)
            score = sbert_similarity(model, s, ner)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "ner": ner,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "ner": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)

Processing: 100%|██████████| 50/50 [00:38<00:00,  1.30it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

In [ ]:
def gpt_negation(sentence: str) -> str:
    prompt = PROMPT_NEGATION.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())

In [8]:
if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            negated = gpt_negation(s)
            score = sbert_similarity(model, s, negated)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "negated": negated,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "negated": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)

Processing: 100%|██████████| 50/50 [00:53<00:00,  1.07s/it]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห